In [ ]:
# Pop2Piano - Fixed & Simplified

This notebook uses a patched version of Pop2Piano that works on modern Colab environments.

## Key Fixes:
- ✅ **Uses your updated GitHub repo with Python 3.13 compatibility**
- ✅ **Fixed all dependency issues** (numpy, typing.io, etc.)
- ✅ **Simplified installation** to use updated `requirements.txt`
- ✅ **No more complex workarounds** for essentia or numpy constraints


In [ ]:
#@title Setup Environment (Simplified)
#@markdown This cell clones your fixed repository and installs the dependencies correctly.

import subprocess
import os

print("🔄 Cloning your fixed repository...")
# Clone YOUR repository with the fixes, not the original one
subprocess.run(['git', 'clone', 'https://github.com/vincefav/pop2piano-colab.git'], check=True)
os.chdir('pop2piano-colab')
print("✅ Cloned successfully!")

print("🔄 Installing system dependencies...")
subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
subprocess.run(['apt-get', 'install', '-y', 'fluidsynth'], check=True, capture_output=True)
print("✅ System dependencies installed!")

print("🔄 Installing Python dependencies...")
# Install numpy first (modern version), then our fixed requirements.txt
subprocess.run(['pip', 'install', 'numpy>=2'], check=True)
subprocess.run(['pip', 'install', '-r', 'requirements.txt'], check=True)
print("✅ Python dependencies installed!")

print("🔄 Downloading model checkpoint...")
if not os.path.exists('model-1999-val_0.67311615.ckpt'):
    subprocess.run(['wget', '-O', 'model-1999-val_0.67311615.ckpt', 
                   'https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt'], 
                   check=True)
print("✅ Model downloaded!")

print("\n🎉 Setup complete! Ready to generate piano covers.")


In [ ]:
#@title Test Installation
#@markdown Run this to verify everything is working correctly.

import sys
print(f"Python version: {sys.version}")
print("Testing imports...\n")

# Test core dependencies
try:
    import numpy as np
    print(f"✅ numpy {np.__version__}")
except Exception as e:
    print(f"❌ numpy: {e}")

try:
    import torch
    print(f"✅ torch {torch.__version__}")
except Exception as e:
    print(f"❌ torch: {e}")

try:
    import librosa
    print(f"✅ librosa {librosa.__version__}")
except Exception as e:
    print(f"❌ librosa: {e}")

# Test our fixed modules
try:
    from transformer_wrapper import TransformerWrapper
    print("✅ transformer_wrapper.TransformerWrapper")
except Exception as e:
    print(f"❌ transformer_wrapper.TransformerWrapper: {e}")

try:
    from midi_tokenizer import MidiTokenizer
    print("✅ midi_tokenizer.MidiTokenizer")
except Exception as e:
    print(f"❌ midi_tokenizer.MidiTokenizer: {e}")

try:
    from preprocess.beat_quantizer import extract_rhythm
    print("✅ preprocess.beat_quantizer (with fallbacks)")
except Exception as e:
    print(f"❌ preprocess.beat_quantizer: {e}")

print("\n🎉 Import testing complete!")


In [ ]:
#@title Generate Piano Cover
#@markdown Paste a YouTube URL here and run the cell to generate piano covers.

YOUTUBE_URL = "https://www.youtube.com/watch?v=dQw4w9WgXcQ" #@param {type:"string"}
NUM_COMPOSERS = 3 #@param {type:"integer"}

import sys
# Add current directory to Python path
if '.' not in sys.path:
    sys.path.insert(0, '.')

from main_no_essentia import main
from argparse import Namespace

# Create arguments for the main function
args = Namespace(
    youtube_url=YOUTUBE_URL,
    audio_file=None,
    start_composer=1,
    end_composer=NUM_COMPOSERS + 1  # Generate the specified number of composers
)

print(f"▶️ Starting generation for: {YOUTUBE_URL}")
print(f"📝 Generating {NUM_COMPOSERS} composer variations...")
main(args)
print("\n✅ Generation complete! Check the file browser on the left for your ZIP file.")


In [ ]:
#@title Generate from Local Audio File (Optional)
#@markdown Upload an audio file and use this cell instead of the YouTube option.

from google.colab import files
import os

print("📁 Upload your audio file:")
uploaded = files.upload()

if uploaded:
    audio_filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {audio_filename}")
    
    NUM_COMPOSERS = 3 #@param {type:"integer"}
    
    from main_no_essentia import main
    from argparse import Namespace
    
    args = Namespace(
        youtube_url=None,
        audio_file=audio_filename,
        start_composer=1,
        end_composer=NUM_COMPOSERS + 1
    )
    
    print(f"▶️ Starting generation for: {audio_filename}")
    print(f"📝 Generating {NUM_COMPOSERS} composer variations...")
    main(args)
    print("\n✅ Generation complete! Check the file browser for your ZIP file.")
else:
    print("❌ No file uploaded.")


In [ ]:
#@title Download Generated Files\n#@markdown Run this cell if you can't see your ZIP file in the file browser\n\nimport os\nimport glob\nfrom google.colab import files\n\n# Look for ZIP files in the current directory\nzip_files = glob.glob(\"*.zip\")\n\nif zip_files:\n    print(\"📁 Found ZIP files:\")\n    for zip_file in zip_files:\n        file_size = os.path.getsize(zip_file)\n        print(f\"   - {zip_file} ({file_size:,} bytes)\")\n    \n    print(\"\\n💾 Choose a file to download:\")\n    \n    # If there's only one ZIP file, download it automatically\n    if len(zip_files) == 1:\n        zip_file = zip_files[0]\n        print(f\"📥 Downloading {zip_file}...\")\n        files.download(zip_file)\n    else:\n        # Let user choose which file to download\n        for i, zip_file in enumerate(zip_files, 1):\n            print(f\"   {i}. {zip_file}\")\n        \n        print(\"\\n💡 To download a specific file, run:\")\n        print(\"   files.download('filename.zip')\")\nelse:\n    print(\"❌ No ZIP files found in the current directory.\")\n    print(\"\\n🔍 Current directory contents:\")\n    for item in os.listdir('.'):\n        if os.path.isfile(item):\n            size = os.path.getsize(item)\n            print(f\"   📄 {item} ({size:,} bytes)\")\n        else:\n            print(f\"   📁 {item}/\")"
